In [ ]:
# distill LLAMA options:
# 1. distill using logits: use an A100, load teacher model and get logits
# 2. distill step by step: use deepseek api and do CoT distill



In [6]:
# install
!pip install -q huggingface_hub
!pip install -q transformers==4.50.0
!pip install -q bitsandbytes==0.45.4


# import
from huggingface_hub import login
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

# Log in to Hugging Face
from google.colab import userdata
my_hugging_face_token = userdata.get('huggingface_token')
login(token=my_hugging_face_token)

# get data from Teacher

In [11]:
# Load with 4-bit quantization

model_id = "meta-llama/Llama-3.3-70B-Instruct"
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="cuda",  # Automatically places layers on GPU/CPU
    #load_in_4bit=True,  # 4-bit quantization
    torch_dtype=torch.float16,  # Optional: mixed precision
    quantization_config=BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.float16,  # Compute dtype
        bnb_4bit_quant_type="nf4",  # NormalFloat4 quantization
        bnb_4bit_use_double_quant=True,  # Nested quantization for better efficiency
    )
)

tokenizer = AutoTokenizer.from_pretrained(model_id)



Loading checkpoint shards:   0%|          | 0/30 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 1.96 GiB. GPU 0 has a total capacity of 39.56 GiB of which 1.73 GiB is free. Process 12055 has 37.82 GiB memory in use. Of the allocated memory 36.83 GiB is allocated by PyTorch, and 497.87 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [12]:
# Inference
input_text = "Explain quantum computing in simple terms:"
inputs = tokenizer(input_text, return_tensors="pt").to("cuda")

# Generate text
outputs = model.generate(
    **inputs,
    max_new_tokens=300,  # Control response length
    temperature=0.7,     # Less deterministic outputs
    do_sample=True       # Enable sampling
)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Explain quantum computing in simple terms: what it is, how it works, and its potential impact on our lives.
Quantum computing is a new way of processing information that's different from the classical computers we use today. In simple terms, quantum computing is a type of computing that uses the principles of quantum mechanics to perform calculations and operations on data.

**What is quantum computing?**

Classical computers use "bits" to store and process information, which can only be in one of two states: 0 or 1. Quantum computers, on the other hand, use "qubits" (quantum bits), which can exist in multiple states simultaneously, like 0, 1, and both 0 and 1 at the same time. This property, called superposition, allows quantum computers to process a vast number of possibilities simultaneously, making them incredibly powerful.

**How does it work?**

Quantum computers use a combination of quantum mechanics and advanced algorithms to perform calculations. Here's a simplified overview:


In [ ]:
# data structure

# Q | A (405B) | A (70B) | logits (70B)


# get data loop